In [138]:
import pandas as pd
import s3fs
import folium


In [139]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [140]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [141]:
df = pd.read_csv('post_processed_listings.csv')

In [142]:
df.head()

,id,Type,Nb_chambre,loc,surface,prix
0,apimo-85473173,Appartement,1,Châtelet,29,364000
1,hektor-Les-4-Quartiers-383,Appartement,1,Châtelet,23,301000
2,hektor-PARISLUXURYHOMES-4007,Appartement,5,Palais Royal,106,2490000
3,ag754594-448898324,Studio,1,Saint-Germain,27,365000
4,keller-williams-1-34_1_53-180645,Appartement,2,Vendôme,67,1190000


In [143]:
df.shape

(8830, 6)

In [152]:
# Ajouter ", Paris" uniquement si "Paris" n'est pas déjà présent dans la colonne loc
df['loc'] = df['loc'].apply(lambda x: x if 'Paris' in x else x + ", Paris")

# Vérification
df.head()


,id,Type,Nb_chambre,loc,surface,prix
0,apimo-85473173,Appartement,1,"Châtelet, Paris",29,364000
1,hektor-Les-4-Quartiers-383,Appartement,1,"Châtelet, Paris",23,301000
2,hektor-PARISLUXURYHOMES-4007,Appartement,5,"Palais Royal, Paris",106,2490000
3,ag754594-448898324,Studio,1,"Saint-Germain, Paris",27,365000
4,keller-williams-1-34_1_53-180645,Appartement,2,"Vendôme, Paris",67,1190000


In [153]:
## Reprise du code de MAXIME pour avoir les même notations ##

# Uniformiser le format des localisation
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 10)
df_ = df.copy()
df_[['parentheses']] = df_['loc'].str.extract(r'\((.*?)\)')
df_['concat'] = df_['parentheses'].fillna('')
df_['loc'] = df_['concat'].where(df_['parentheses'].notna(), df_['loc'])
df_.drop(columns=['parentheses', 'concat'], inplace=True)

df_[['parentheses']] = df_['loc'].str.extract(r'^(.*?)\s*\s-')
df_['concat'] = df_['parentheses'].fillna('')
df_['loc'] = df_['concat'].where(df_['parentheses'].notna(), df_['loc'])
df_.drop(columns=['parentheses', 'concat'], inplace=True)

df_zones=df_["loc"].drop_duplicates().copy()
df=df_.copy()

In [154]:
from geopy.geocoders import Nominatim
from functools import partial
geolocator = Nominatim(user_agent="testing")
geocode = partial(geolocator.geocode, language="fr")
geocode_with_paris = lambda query: geolocator.geocode("%s, Paris FR" % query)

In [155]:
# Initialiser une liste vide pour stocker les zones
tableau_zones = []

# Parcourir les zones et les ajouter au tableau
for zone in df_zones:
    print(zone)  # Affiche chaque zone
    tableau_zones.append(zone)  # Ajoute chaque zone au tableau

# Afficher le tableau final après la boucle
print("\nTableau des zones :")
print(tableau_zones)

Châtelet, Paris
Palais Royal, Paris
Saint-Germain, Paris
Vendôme, Paris
Gaillon, Paris
Saint-Thomas d'Aquin, Paris
75001 Paris 1er
Sentier, Paris
Montorgueil, Paris
75002 Paris 2e
Arts-et-Métiers, Paris
Sainte-Avoye, Paris
Archives, Paris
75003 Paris 3e
Enfants Rouges, Paris
République, Paris
Saint-Merri, Paris
Arsenal, Paris
75004 Paris 4e
Les Iles, Paris
Saint-Gervais, Paris
75000 Paris
Monge, Paris
Jardin des Plantes, Paris
Val de Grâce, Paris
Sorbonne, Paris
Rennes, Paris
Montparnasse, Paris
Saint-Germain des Près, Paris
Odéon, Paris
Saint-Placide, Paris
Notre Dame des Champs, Paris
75006 Paris 6e
Saint-Michel, Paris
Hoche Friedland, Paris
Elysée, Paris
Monceau, Paris
Triangle d'Or, Paris
Europe, Paris
Saint-Philippe du Roule, Paris
Mairie, Paris
75008 Paris 8e
Clichy, Paris
Grands Boulevards, Paris
Lorette, Paris
Opéra, Paris
Trudaine, Paris
Porte Saint-Martin, Paris
75009 Paris 9e
Saint-Vincent de Paul, Paris
Château d'Eau, Paris
Goncourt, Paris
Canal Saint-Martin, Paris
Louis Bl

In [157]:
len(tableau_zones)

140

In [158]:
from geopy.geocoders import Nominatim
import time

# Créer un objet géocodeur Nominatim pour extraire la longitude et la latitude de chaque zone
geolocator = Nominatim(user_agent="my_geocoder")

# Fonction pour obtenir les coordonnées géographiques (longitude et latitude) d'une adresse
def obtenir_coordonnees(tableau_zones):
    try:
        location = geolocator.geocode(tableau_zones, timeout=5)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Erreur pour {tableau_zones}: {e}")
        return None, None


# Créer un dictionnaire pour stocker les adresses et leurs coordonnées géographiques
coordonnees_dict = {}

for zone in tableau_zones:
    lat, lon = obtenir_coordonnees(zone)  # Passer 'zone_paris' à la fonction
    coordonnees_dict[zone] = (lat, lon)   # Utiliser 'zone_paris' comme clé
    
    # Pause pour éviter un blocage par le serveur
    time.sleep(1)

# Créer une liste pour stocker les informations des adresses
coordonnees = []

# Afficher et stocker les coordonnéesde toutes les zones
print("\nCoordonnées des 3 premières adresses :")
for zone, coord in coordonnees_dict.items():
    print(f"Adresse: {zone}, Latitude: {coord[0]}, Longitude: {coord[1]}")
    coordonnees.append({"adresse": zone, "latitude": coord[0], "longitude": coord[1]})  # Stocker les données

# Afficher le contenu de coordonnees
print("\nContenu de coordonnees2 :")
for item in coordonnees:
    print(item)


Coordonnées des 3 premières adresses :
Adresse: Châtelet, Paris, Latitude: 48.8597916, Longitude: 2.3466756
Adresse: Palais Royal, Paris, Latitude: 48.863584700000004, Longitude: 2.3362042200938715
Adresse: Saint-Germain, Paris, Latitude: 48.85655255, Longitude: 2.333331104782813
Adresse: Vendôme, Paris, Latitude: 48.86690005, Longitude: 2.3285489140926545
Adresse: Gaillon, Paris, Latitude: 48.869135150000005, Longitude: 2.332908770335507
Adresse: Saint-Thomas d'Aquin, Paris, Latitude: 48.85424375, Longitude: 2.3268829064838443
Adresse: 75001 Paris 1er, Latitude: 48.8646144, Longitude: 2.334396
Adresse: Sentier, Paris, Latitude: 48.8675428, Longitude: 2.3465779
Adresse: Montorgueil, Paris, Latitude: 48.8641678, Longitude: 2.3467038
Adresse: 75002 Paris 2e, Latitude: 48.868743, Longitude: 2.341688
Adresse: Arts-et-Métiers, Paris, Latitude: 48.8654718, Longitude: 2.3561116
Adresse: Sainte-Avoye, Paris, Latitude: 48.862699750000004, Longitude: 2.354135471358302
Adresse: Archives, Paris, 

In [159]:
coordonnees

[{'adresse': 'Châtelet, Paris',
  'latitude': 48.8597916,
  'longitude': 2.3466756},
 {'adresse': 'Palais Royal, Paris',
  'latitude': 48.863584700000004,
  'longitude': 2.3362042200938715},
 {'adresse': 'Saint-Germain, Paris',
  'latitude': 48.85655255,
  'longitude': 2.333331104782813},
 {'adresse': 'Vendôme, Paris',
  'latitude': 48.86690005,
  'longitude': 2.3285489140926545},
 {'adresse': 'Gaillon, Paris',
  'latitude': 48.869135150000005,
  'longitude': 2.332908770335507},
 {'adresse': "Saint-Thomas d'Aquin, Paris",
  'latitude': 48.85424375,
  'longitude': 2.3268829064838443},
 {'adresse': '75001 Paris 1er', 'latitude': 48.8646144, 'longitude': 2.334396},
 {'adresse': 'Sentier, Paris', 'latitude': 48.8675428, 'longitude': 2.3465779},
 {'adresse': 'Montorgueil, Paris',
  'latitude': 48.8641678,
  'longitude': 2.3467038},
 {'adresse': '75002 Paris 2e', 'latitude': 48.868743, 'longitude': 2.341688},
 {'adresse': 'Arts-et-Métiers, Paris',
  'latitude': 48.8654718,
  'longitude': 2.3

In [160]:
# Convertir les coordonnées en DataFrame
coordonnees = pd.DataFrame(coordonnees)

# Supprimer les lignes contenant des NaN pour 'latitude' ou 'longitude'
coordonnees = coordonnees.dropna(subset=['latitude', 'longitude'])

# Vérification des résultats
print(coordonnees)

                  adresse   latitude  longitude
0         Châtelet, Paris  48.859792   2.346676
1     Palais Royal, Paris  48.863585   2.336204
2    Saint-Germain, Paris  48.856553   2.333331
3          Vendôme, Paris  48.866900   2.328549
4          Gaillon, Paris  48.869135   2.332909
..                    ...        ...        ...
135       75020 Paris 20e  48.865042   2.398929
136   Ménilmontant, Paris  48.865042   2.398929
137  Père Lachaise, Paris  48.861130   2.394009
138         Plaine, Paris  48.850152   2.402190
139   Saint-Blaise, Paris  48.858567   2.406324

[138 rows x 3 columns]


TypeError: string indices must be integers, not 'str'

In [163]:
len(coordonnees)

138

In [164]:
len(coordonnees_filtrees) # on verifie qu'on a bien une coordonnées en moins

139

In [165]:
# Diviser les adresses en fonction des noms spécifiques ou vagues
noms_specifiques = []
noms_vagues = []

for item in coordonnees_filtrees:
    if any(char.isdigit() for char in item['adresse']):  # Vérifier si l'adresse contient des chiffres (arrondissements)
        noms_vagues.append(item)
    else:
        noms_specifiques.append(item)

print(noms_specifiques)
print(noms_vagues)

[{'adresse': 'Châtelet, Paris', 'latitude': 48.8597916, 'longitude': 2.3466756}, {'adresse': 'Palais Royal, Paris', 'latitude': 48.863584700000004, 'longitude': 2.3362042200938715}, {'adresse': 'Saint-Germain, Paris', 'latitude': 48.85655255, 'longitude': 2.333331104782813}, {'adresse': 'Vendôme, Paris', 'latitude': 48.86690005, 'longitude': 2.3285489140926545}, {'adresse': 'Gaillon, Paris', 'latitude': 48.869135150000005, 'longitude': 2.332908770335507}, {'adresse': "Saint-Thomas d'Aquin, Paris", 'latitude': 48.85424375, 'longitude': 2.3268829064838443}, {'adresse': 'Sentier, Paris', 'latitude': 48.8675428, 'longitude': 2.3465779}, {'adresse': 'Montorgueil, Paris', 'latitude': 48.8641678, 'longitude': 2.3467038}, {'adresse': 'Arts-et-Métiers, Paris', 'latitude': 48.8654718, 'longitude': 2.3561116}, {'adresse': 'Sainte-Avoye, Paris', 'latitude': 48.862699750000004, 'longitude': 2.354135471358302}, {'adresse': 'Archives, Paris', 'latitude': 48.8596256, 'longitude': 2.362541376680502}, {

In [166]:
### CREATION DE LA CARTE ###

# Initialiser la carte centrée sur Paris
carte = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Ajouter des points sur la carte pour les adresses avec coordonnées
for lieu in noms_specifiques :          # J'affiche seulement les adresses spécifiques (pas les vagues pour eviter tout problème parce qu'elles n'apparaissent pas au bond endroit sur la carte...)
    if lieu["latitude"] is not None and lieu["longitude"] is not None:
        folium.Marker(
            location=[lieu["latitude"], lieu["longitude"]],
            popup=lieu["adresse"],      # Texte affiché lorsqu'on clique sur le point
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(carte)

# Sauvegarder la carte dans un fichier HTML
carte.save("carte_paris_points.html")
print("Carte créée : carte_paris_points.html")

Carte créée : carte_paris_points.html


In [167]:
## Affichage de la carte
carte

In [168]:
# Regrouper par 'loc' et calculer la moyenne des prix et des surfaces
resultat = df.groupby('loc', as_index=False).agg(
    moyenne_prix=('prix', 'mean'),        # Moyenne des prix
    moyenne_surface=('surface', 'mean')  # Moyenne des surfaces
)

# Calculer la moyenne des prix au m²
resultat['prix_m2'] = resultat['moyenne_prix'] / resultat['moyenne_surface']

# Supprimer les colonnes intermédiaires si elles ne sont pas nécessaires
resultat = resultat[['loc', 'prix_m2']]  # Garder seulement loc et prix_m2

# Vérification des résultats
print(resultat)

                         loc       prix_m2
0                75000 Paris   4456.521739
1            75001 Paris 1er  20959.770115
2             75002 Paris 2e  12623.655914
3             75003 Paris 3e  10774.245115
4             75004 Paris 4e  21924.863388
..                       ...           ...
135      Val de Grâce, Paris  13455.888651
136  Vallée de Fecamp, Paris   8803.143394
137         Vaugirard, Paris  10369.205115
138           Vendôme, Paris  17642.301545
139            Violet, Paris  11067.360796

[140 rows x 2 columns]


In [ ]:
# Convertir les coordonnées en DataFrame
coord_df = pd.DataFrame(coordonnees)

# Fusionner les deux DataFrames
resultat = resultat.merge(coord_df[['adresse', 'latitude', 'longitude']], 
                          left_on='loc', 
                          right_on='adresse', 
                          how='left')

# Supprimer la colonne 'adresse' qui est redondante après la fusion
resultat.drop(columns=['adresse'], inplace=True)

# Vérification des résultats
print(resultat)

In [174]:
# Supprimer les lignes contenant des NaN dans le DataFrame 'resultat'
resultat = resultat.dropna()

# Vérification des résultats
print("DataFrame après suppression des NaN :")
print(resultat)

DataFrame après suppression des NaN :
                         loc       prix_m2   latitude  longitude
0                75000 Paris   4456.521739  48.858890   2.320041
1            75001 Paris 1er  20959.770115  48.864614   2.334396
2             75002 Paris 2e  12623.655914  48.868743   2.341688
3             75003 Paris 3e  10774.245115  48.864212   2.360936
4             75004 Paris 4e  21924.863388  48.856202   2.355619
..                       ...           ...        ...        ...
135      Val de Grâce, Paris  13455.888651  48.842243   2.343892
136  Vallée de Fecamp, Paris   8803.143394  48.834991   2.398645
137         Vaugirard, Paris  10369.205115  48.839587   2.301280
138           Vendôme, Paris  17642.301545  48.866900   2.328549
139            Violet, Paris  11067.360796  48.844800   2.289992

[138 rows x 4 columns]


In [189]:
import matplotlib.pyplot as plt

# Palette de couleurs basée sur des teintes rouges
cmap = plt.cm.Reds  # Palette Reds pour des teintes rouges

# Normalisation des couleurs en fonction de 'prix_m2'
norm = plt.Normalize(vmin=resultat['prix_m2'].min(), vmax=resultat['prix_m2'].max())

# Fonction pour mapper les prix/m² à une couleur
def prix_m2_to_color(prix_m2):
    rgba = cmap(norm(prix_m2))
    return f"rgba({int(rgba[0]*255)}, {int(rgba[1]*255)}, {int(rgba[2]*255)}, {rgba[3]})"

# Création de la carte
carte = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Ajout des points
for _, row in resultat.iterrows():
    couleur = prix_m2_to_color(row['prix_m2'])  # Couleur basée sur le prix/m²
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=7,  # Taille des points
        color='black',  # Contour noir
        fill=True,
        fill_color=couleur,  # Remplissage rouge
        fill_opacity=0.9,
        popup=f"{row['loc']}<br>Prix/m² : {row['prix_m2']:.2f} €"
    ).add_to(carte)

# Sauvegarder la carte
carte.save("carte_prix_m2_rouge.html")
print("Carte créée : carte_prix_m2_rouge.html")

Carte créée : carte_prix_m2_rouge.html


In [ ]:
# Les prix bas : en rouge clair.
# Les prix élevés : en rouge foncé.

carte